# Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1237]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
#определяем ключевые слова для дальнейшего поиска по ним статей
KEYWORDS = ['science','linux','azure','js','страх']
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
# извлекаем посты
posts = soup.find_all('div', class_='tm-article-snippet')
habr_frame = pd.DataFrame()
habr_frame_new = pd.DataFrame()
# проходимся в цикле по всем статьям
for post in posts:
    date = post.find('span', class_='tm-article-snippet__datetime-published')
    head = post.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2')
    #переходим по полученой нами ссылке статьи и вытягиваем оттуда текст статьи
    link_full= 'https://habr.com' + head.a.get('href') 
    req_link = requests.get(link_full)
    soup_link = BeautifulSoup(req_link.text, 'html.parser')
    #hubs -это текст статьи с тегами (может быть оформлена двумя классами ...version-1 и version-2)
    hubs1 = soup_link.find_all('div', class_='article-formatted-body article-formatted-body_version-1') 
    hubs= hubs1 + soup_link.find_all('div', class_='article-formatted-body article-formatted-body_version-2')
    #разбиваем на слова статьи 
    for hub in hubs: 
        hub_lower = hub.text.lower()
        hubs_list = hub_lower.split(' ')
        #сравниваем каждое слово в превьюшке статьи с нашим словариком
        for word in hubs_list:
            if word in KEYWORDS:
                date_short = pd.to_datetime(date.time.get('title')).date()
                habr_frame['дата']= [date_short]
                link_full= 'https://habr.com' + head.a.get('href')    
                habr_frame['заголовок']= [head.text] 
                habr_frame['ссылка']= [link_full] 
                #вытягиваем из soup текст статьи по тегам 'p'
                text_draft = soup_link.find_all('p')
                #поблочно перебираем статью и отчищаем от тегов для сохранения в датафрейме
                text_all = ''
                for word in text_draft: 
                    text_all = text_all+word.text
                habr_frame['текст статьи']= [text_all]
                habr_frame_new = pd.concat([habr_frame_new,habr_frame])
#удаляем дубликаты статей и выводим датафрейм
habr_frame_new.drop_duplicates('заголовок', keep="first")

,дата,заголовок,ссылка,текст статьи
0,2021-07-22,За что я люблю ассемблер?,https://habr.com/ru/post/569204/,Этой статье уже почти 3 года. Однако сегодня я...
0,2021-07-22,Зарплаты в Data Science будут расти несмотря н...,https://habr.com/ru/company/skillfactory/blog/...,Оригинал первой части перевода вы найдёте здес...
0,2021-07-22,Опыт установки современного дистрибутива Linux...,https://habr.com/ru/post/569198/,"Привет, Хабр!На reddit, в сообществах посвяще..."
0,2021-07-22,Создание инцидента через бота MS Teams,https://habr.com/ru/post/569166/,Повышение осведомленности работников в области...
0,2021-07-22,Golang+FFmpeg,https://habr.com/ru/post/569162/,Долго искал более-менее живую Golang-библиотек...
0,2021-07-22,"Сейчас я вас научу, как именно бояться стомато...",https://habr.com/ru/company/belayaraduga/blog/...,Главный врач


# Задание 2.

Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>




In [1136]:
import requests
import time


URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
HEADERS = {
    
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'ru-RU,ru;q=0.9,pl-PL;q=0.8,pl;q=0.7,en-US;q=0.6,en;q=0.5',
            'Connection': 'keep-alive',
            'Content-Length': '31',
            'Content-Type': 'application/json;charset=UTF-8',
            'Host': 'identityprotection.avast.com',
            'Origin': 'https://www.avast.com',
            'Referer': 'https://www.avast.com/',
            'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
            'sec-ch-ua-mobile': '?0',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-site',
            'User-Agent': 'Mozilla/4.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
            'Vaar-Header-App-Build-Version': '1.0.0',
            'Vaar-Header-App-Product': 'hackcheck-web-avast',
            'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
            'Vaar-Version': '0'
}
EMAILS = {
    'emailAddresses': ['xxx@x.ru']
}
EMAIL = ['xxx@x.ru','yyy@y.com']
leaks_frame = pd.DataFrame()
leaks_frame_new = pd.DataFrame()
for i in range(len(EMAIL)):
    EMAILS['emailAddresses'] = [EMAIL[i]]
    print(EMAILS)
    r = requests.post(URL, headers=HEADERS, json=EMAILS)
    time.sleep(5)
    #print(r)
    respond = r.json()

    for x in respond['breaches'].keys():
        leaks_frame['дата утечки']= [respond['breaches'][x]['publishDate']] 
        leaks_frame['источник утечки']= [respond['breaches'][x]['site']] 
        leaks_frame['описание утечки']= [respond['breaches'][x]['description']]
        leaks_frame_new = pd.concat([leaks_frame_new,leaks_frame])
leaks_frame_new 

{'emailAddresses': ['xxx@x.ru']}
{'emailAddresses': ['yyy@y.com']}


,дата утечки,источник утечки,описание утечки
0,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
0,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
0,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,2016-10-21T00:00:00Z,linkedin.com,"In 2012, online professional networking platfo..."
0,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
0,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
